# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This version includes a Gradio UI, streaming, system prompt expertise modes, and model switching.

In [7]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [8]:
# constants
OPENAI_MODELS = ["gpt-4o-mini", "gpt-4o", "gpt-5"]
MODEL_LLAMA = "llama3.2"
ALL_MODELS = OPENAI_MODELS + [MODEL_LLAMA]
DEFAULT_MODEL = OPENAI_MODELS[0]
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [9]:
load_dotenv(".env", override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is missing. Add it to a .env file.")

openai_client = OpenAI(api_key=openai_api_key)
ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

In [10]:
expertise_modes = {
    "General Tutor": "You are a helpful technical tutor. Explain clearly with simple examples.",
    "AI Tools Mentor": "You are an AI tools mentor. Explain practical steps, trade-offs, and one beginner mistake to avoid.",
    "Interview Coach": "You are a technical interview coach. Explain clearly, then end with one short follow-up practice question."
}

In [11]:
def respond(message, history, model, expertise):
    history = history or []

    if not message.strip():
        yield "", history
        return

    client = openai_client if model in OPENAI_MODELS else ollama_client
    system_prompt = expertise_modes[expertise]

    chat_history = []
    for turn in history:
        if isinstance(turn, dict):
            role = turn.get("role")
            content = turn.get("content")
            if role in ("user", "assistant") and content:
                chat_history.append({"role": role, "content": content})
        elif isinstance(turn, (list, tuple)) and len(turn) == 2:
            user_msg, assistant_msg = turn
            if user_msg:
                chat_history.append({"role": "user", "content": user_msg})
            if assistant_msg:
                chat_history.append({"role": "assistant", "content": assistant_msg})

    api_messages = [{"role": "system", "content": system_prompt}] + chat_history
    api_messages.append({"role": "user", "content": message})

    partial = ""
    stream = client.chat.completions.create(
        model=model,
        messages=api_messages,
        stream=True,
    )

    base_history = chat_history + [{"role": "user", "content": message}]
    for chunk in stream:
        token = chunk.choices[0].delta.content or ""
        if token:
            partial += token
            yield "", base_history + [{"role": "assistant", "content": partial}]

In [12]:

with gr.Blocks() as demo:
    gr.Markdown("## Week 2 Technical Q&A Assistant")
    gr.Markdown("Ask a technical question, switch model and expertise mode, then chat.")

    with gr.Row():
        with gr.Column(scale=4, min_width=780, elem_id="main-pane"):
            chatbot = gr.Chatbot(label="Assistant", height=520)

            with gr.Row(elem_id="input-row"):
                msg = gr.Textbox(
                    placeholder="Ask a technical question...",
                    label="Message",
                    lines=2,
                    max_lines=5,
                    scale=8,
                )
                send_btn = gr.Button("Send", variant="primary", scale=2, min_width=140)

        with gr.Column(scale=2, min_width=300, elem_id="left-sidebar"):
            gr.Markdown("### Settings")
            model_dropdown = gr.Dropdown(
                choices=ALL_MODELS,
                value=DEFAULT_MODEL,
                label="AI Model",
                interactive=True,
            )
            mode_dropdown = gr.Dropdown(
                choices=list(expertise_modes.keys()),
                value="AI Tools Mentor",
                label="Mode",
                interactive=True,
            )
            clear_btn = gr.Button("Clear Chat")

    send_btn.click(
        fn=respond,
        inputs=[msg, chatbot, model_dropdown, mode_dropdown],
        outputs=[msg, chatbot],
    )
    msg.submit(
        fn=respond,
        inputs=[msg, chatbot, model_dropdown, mode_dropdown],
        outputs=[msg, chatbot],
    )
    clear_btn.click(lambda: ("", []), outputs=[msg, chatbot], queue=False)

demo.launch(
    theme=gr.themes.Soft(),
    css=".gradio-container {max-width: 1400px !important; width: 96vw !important; margin: 0 auto;} #right-sidebar {border-left: 1px solid rgba(255,255,255,0.10); padding-left: 14px;} #input-row {align-items: end;}"
)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
